In [91]:
from bs4 import BeautifulSoup
import requests
import re
import time
import random
from random import randint

# Url of website
url='https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411'
# I slow down the frequency of requests to avoid being identified and therefore ban from the site
time.sleep(random.uniform(1.0, 2.0))
# I send my HTTP request with a "GET" to the site server to identify in the url
r = requests.get(url)
# I display the requested url and the return of the server
print(url, r.status_code)
# I ask beautifulSoup to keep in a soup variable the web page to scrape (url) an html script
soup = BeautifulSoup(r.content,'lxml')

# initialise the output dictionary
property_dict = {}

# find useful data in every row element
for elem in soup.find_all('tr'):
    # when it has a header child
    if elem.th:
        # and this child has only one string as child, then this is info to store
        if elem.th.string:
            info_to_store = elem.th.string.strip()
            # price info is an exception scraping case
            if info_to_store == "Price":
                r = re.compile("(\d{5,})\s€")                
                for descendant in elem.td.descendants:
                    if r.search(str(descendant)):
                        property_dict["Price"] = int(r.search(str(descendant))[1])
                        break
            # Address info is an exception scraping case, it is spread over different children of the data child
            elif info_to_store == "Address":
                address_list = [str(address_line).strip() for address_line in elem.td.contents]
                value_to_store = ''.join(address_list).replace("\n",",").replace("<br/>",",")
                property_dict[info_to_store] = value_to_store
            # The standard case is that values are present in the first child of the data child
            else:
                value_to_store = elem.td.contents[0].strip()
                if value_to_store.isnumeric():
                    property_dict[info_to_store] = int(value_to_store)
                else:
                    property_dict[info_to_store] = value_to_store

# store the postcode from url
property_dict["postcode"] = re.findall('/[0-9]{4}/', url)[0].replace('/', '')
                    
print(property_dict)
print(len(property_dict))
                

https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411 200
{'Available as of': 'Upon exchange of deeds', 'Building condition': 'Good', 'Facades': 3, 'Living area': 70, 'Kitchen type': 'Semi equipped', 'Bedrooms': 2, 'Bedroom 1 surface': 20, 'Bedroom 2 surface': 14, 'Bathrooms': 1, 'Toilets': 1, 'Basement': 'Yes', 'Surface of the plot': 93, 'Connection to sewer network': 'Connected', 'Gas, water & electricity': 'Yes', 'Terrace': 'Yes', 'Primary energy consumption': 340, 'Energy class': 'D', 'Reference number of the EPC report': 20170928019834, 'CO² emission': 'Not specified', 'Yearly theoretical total energy consumption': '26952 kWh/year', 'Heating type': 'Gas', 'Double glazing': 'Yes', 'Price': 129000, 'Cadastral income': '', 'Address': "Boulevard d'Avroy 238B,4000,                                                                                                                            - Liège", 'Website': '', 'External reference': 4045180}
27
